In [10]:
import string
import sys
import io 
import nltk
import pandas as pd
import numpy as np

# Stopwords
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import stopwords 
nltk.download('stopwords')
stops = set(stopwords.words('italian'))
nltk_stopwords = nltk.corpus.stopwords.words('italian')
# Valutare di aggiungere eventuali forme arcariche delle stopwords italiane, o se c'è necessità di estendere questa lista
  
# Divido il testo in frasi in base ai punti
nltk.download('punkt')

# Tokenizer
from nltk.tokenize import word_tokenize 

# Per il discorso lemmatizzazione dobbiamo valutare come muoverci con l'italiano

# Lemmatization
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Contatore parole uniche
from collections import Counter

# Per esplorare risultati
import random

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
!pip install cade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
from gensim.models import Word2Vec
from cade.cade import CADE

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### CREO TUTTO IL NECESSARIO PER TESTARE LE FUNZIONI

In [5]:
# Preprocessing (NO lemmatization)
def preprocessing(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

In [8]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/DataSemanticsProject/Books')

In [9]:
frasi_decameron = preprocessing('Decameron.txt')
frasi_orlando = preprocessing('Orlando furioso.txt')

### DEFINIZIONE FUNZIONI

In [18]:
# Funzione per addestrare n modelli per corpus

def training_W2V(sentences, text, n_mod): #file con frasi del corpus, nome del corpus, numero di modelli da addestrare

  for k in range(n_mod):
    model = Word2Vec(sentences = sentences,
                    #window = 5, default value
                    min_count=10, #not consider word with absolute frequency <10 
                    size=300, #vector size 
                    sg = 1, #skipgram algorithm
                    hs = 0,
                    negative = 5, #negative sampling with 5 noise words
                    workers = 5, #faster process
                    iter = 6 #6 iterations
                    )
  
    model.save(text.lower() + "_" + str(k) + ".model")


# Funzione per addestrare n slice per corpus con CADE

# !cat corpus1.txt corpus2.txt corpus3.txt ... > compass.txt

def training_CADE(texts, n_mod): #lista contenente i corpus usati per creare la compass in ordine, numero di slices da addestrare
                                 #la lista dev'essere composta dal nome esatto del file txt ma senza l'estensione

  aligner = CADE(min_count=10,  
                  size=300,
                  sg = 1, 
                  #hs = 0,
                  ns = 5, 
                  workers = 5,
                  siter = 6)

  for k in range(n_mod):
    aligner.train_compass('compass.txt', overwrite=True)
    for text in texts:
      model_slice = aligner.train_slice(text + ".txt") #per trainare le slice ho bisogno del nome esatto del file .txt con cui ho creato il compasso
      model_slice.save(text + '_cade_' + str(k) + '.model') #qui posso salvare il modello con un nome a piacere

In [20]:
# Funzione per estrarre le n parole più simili ad una parola target, calcolando la similarità media, per un dato numero di modelli creati
# Ipotizzo in questo caso di aver creato un numero n_mod di modelli, così da rendere più stabili i risultati

def similar_words_per_topos(word, n_mod, n_words, text): #parola target da analizzare, numero di modelli da considerare, numero di parole simili da estrarre, corpus da analizzare

  #os.chdir('') #inserire path alla cartella con i corpus
  text_2 = text
  text = text.lower()

  #ottengo gli n modelli del testo considerato
  embeddings = []
  for k in range(n_mod):
    model = Word2Vec.load(text.lower() + '_' + str(k) + '.model') #adattare eventualmente nome e path dei modelli per il caricamento
    embeddings.append(model) #ottengo una lista contenente gli n modelli, precedentemente creati, per il testo desiderato

  #modello per modello, ottengo la lista delle m parole (n_words) più simili al target
  words = []
  for k in range(n_mod):
    for tupla in embeddings[k].wv.most_similar(word)[:n_words]: #l'output di most_similar è una lista di tuple che presenta la parola e il rispettivo valore di similarità
      words.append(tupla[0]) #appendo solamente la parola, senza considerare il valore di similarità rispetto al target

  #creo un dizionario che presenta le parole uniche contenute nella lista "words", e le rispettive frequenze assolute
  c = Counter(words)
  word_frequencies = dict(c)

  #creo un dizionario che contiene la lista di parole uniche in "words" e calcola la similarità media di ciascuna di esse rispetto alla parola target
  init = [0] * len(np.unique(words))
  word_similarities = dict(zip(np.unique(words), init))

  for k in range(n_mod):
    for tupla in embeddings[k].wv.most_similar(word)[:n_words]:
      word_similarities[tupla[0]] += tupla[1]
  for word in np.unique(words):
    word_similarities[word] = word_similarities[word] / word_frequencies[word]

  #trasformo il dizionario con le frequenze assolute e quello con le similarità medie in dataframe, e li unisco
  df_1 = pd.DataFrame(list(word_frequencies.items()), columns=['Words','Abs.frequency'])
  df_2 = pd.DataFrame(list(word_similarities.items()), columns=['Words','Med.similarity'])
  #calcolo anche la similarità normalizzata, nel caso dovesse servie
  normalized_similarity = (df_2['Med.similarity']-df_2['Med.similarity'].min())/(df_2['Med.similarity'].max() - df_2['Med.similarity'].min()) 
  df = pd.merge(df_1, df_2, on = 'Words')
  df['Med.similarity(norm.)'] = normalized_similarity

  #aggiungo la colonna col nome del testo considerato
  df['Text'] = [text_2] * len(df)

  return df

In [36]:
# Funzione per comparare una parola fra embedding diversi

def compare_word_btw_embeddings(word_to_study, n_mod, text1, text2): #parola da studiare, numero di embedding creati e da studiare, testo 1 preso come riferimento (da questo testo
                                                                     # estraggo il vettore della parola target), testo 2 da cui estraggo le parole simili

  #prendo il numero totale di embedding dei due testi considerati
  embeddings1 = []
  for it in range(n_mod):
    model = Word2Vec.load(text1 + '_cade_' + str(it) + '.model') #ho scritto cade perchè tendenzialmente questa funzione sarà usata su embedding allineati
    embeddings1.append(model)

  embeddings2 = []
  for it in range(n_mod):
    model = Word2Vec.load(text2 + '_cade_' + str(it) + '.model')
    embeddings2.append(model)

  #ottengo la lista delle parole più simili alla target per ciascun modello
  words = []
  for k in range(n_mod):
    word_vector = embeddings1[k].wv[word_to_study]
    for tupla in embeddings2[k].wv.most_similar(positive=[word_vector])[:10]: #ho preso arbitrariamente 10 parole, si può modificare a piacimento
      words.append(tupla[0])

  #creo dizionario con parole uniche e relativa frequenza assoluta
  c = Counter(words)
  word_frequencies = dict(c)

  #creo dizionario con parole uniche calcolando la similarità media
  init = [0] * len(np.unique(words))
  word_similarities = dict(zip(np.unique(words), init))

  for k in range(n_mod):
    word_vector = embeddings1[k].wv[word_to_study]
    for tupla in embeddings2[k].wv.most_similar(positive=[word_vector])[:10]:
      word_similarities[tupla[0]] += tupla[1]
  for word in np.unique(words):
    word_similarities[word] = word_similarities[word] / word_frequencies[word]
  
  #genero il dataframe finale
  df = pd.DataFrame(list(word_similarities.items()), columns=['Words','Med.similarity'])
  df = df.sort_values(by=['Med.similarity'], ascending=False)
  df = df.head(n=10)
  return df



  ### DA USARE POI NEL SEGUENTE CICLO
  #for text in corpora:
  #  print('***' + text +'***')
  #  df = compare_word_btw_embeddings('parola', 'testo di riferimento', text)
  #  print(df)

In [ ]:
# Funzione per creare una heatmap di frequenza delle parole più simili a una parola target rispetto a vari corpus (presa da Simo, legata alla funzione precedente)

def freq_heatmap_preparation(column, word, n_mod, n_words): #colonna da considerare nella heatmap (frequenza assoluta o similarità media), parola target, numero di modelli da 
                                                            #considerare, numero di parole da considerare
  dfs_list = []
  corpus_texts = ['SCRIVERE'] #inserire i nomi dei corpus su cui iterare la funzione (VANNO INSERITI MANUALMENTE)

  for text in corpus_texts:
    dfs_list.append(explore_word(word, n_mod, n_words, text)) #richiamo la funzione precedente e appendo i dataframe creati a una lista di dataframes

  result = pd.concat(dfs_list, ignore_index=True) #concateno i dataframes appesi alla lista
  df = result[['Words', column, 'Text']] #riordino a piacimento le colonne
  df_hm = df.pivot_table(index='Text', columns='Words', values=column) #effettuo un pivot per avere i dati pronti da inserire nella heatmap

  return df_hm

In [ ]:
# Funzione per generare la heatmap con i dati preparati con la funzione precedente

def plot_the_heatmap(df_heatmap, word, column): #dataframe preparato con funzione precedente, parola target, colonna analizzata (frequenza assoluta o similarità media)
  #imposto la heatmap (cambiare a piacimento)
  plt.figure(figsize=(20,4))
  cmap = sns.color_palette("Greens", as_cmap=True) 

  if(column == 'Abs.frequency'):
    bounds = [] #inserire manualmente i bounds della heatmap a seconda dei nostri valori di frequenza
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    hm = sns.heatmap(df_heatmap, cmap=cmap, norm=norm, cbar_kws={"shrink": 1.2}, linewidths=1)

  elif((column == 'Med.similarity') or (column == 'Med.similarity(norm.)')):
    hm = sns.heatmap(df_heatmap, cmap=cmap, cbar_kws={"shrink": 1.2}, linewidths=1)

  else:
    print('Colonna sbagliata')
    return

  hm.set_xlabel(xlabel = 'Words', fontsize = 16)
  hm.set_ylabel(ylabel = column , fontsize = 17)
  hm.set_xticklabels(hm.get_xmajorticklabels(), fontsize = 14)
  hm.set_yticklabels(hm.get_ymajorticklabels(), fontsize = 14)
  hm.axes.set_title(column + " INSERIRE TITOLO "+ word, fontsize=20)
  hm.set_facecolor('lightgrey')
  plt.savefig('NOME_FIGURA')

In [47]:
# Funzione per esplorare analogie e calcolare similarità media dati tutti i modelli addestrati per un corpus
# SE NECESSARIO, USARE LA STESSA FUNZIONE ANCHE PER I MODELLI ALLINEATI ADDESTRATI CON CADE, VA SOLO CAMBIATO IL NOME DEL FILE NEL PRIMO for

def compute_analogies(text, n_mod, word1, word2, word3): #corpus su cui eseguire le analogie, numero di modelli addestrati, word1 & word2 positive, word3 negativa
  models = []

  for k in range(n_mod): #inserire manualmente il numero di modelli addestrati 
    model = Word2Vec.load(text+'_'+str(k)+'.model') #sistemare il nome in base a come sono nominati i modelli
    models.append(model) #appendo i modelli ad una lista

  list_tuples = []
  list_words = []

  for i in range(n_mod):
    list_tuples.append(models[i].most_similar(positive=[word1,word2], negative=[word3])) #per tutti i modelli caricati calcolo l'analogia (ho delle tuple con (parola, similarità))
      
  for i in list_tuples:
    for element in i:
      list_words.append(element) #appendo le tuple una per una 

  

  df = pd.DataFrame(list_words, columns=['Words', 'Cosine_similarity']) #creo un dataframe
  df = df.groupby('Words').mean() #raggruppo per parola calcolando la media
  df.sort_values(by=['Cosine_similarity'], inplace = True, ascending = False) #ordino per similarità massima
  return df       

In [50]:
# Funzioni per estrarre sinonimi di una certa parola target, e per estrarre GLI AGGETTIVI più simili a quella parola (o insieme di parole, considerando i sinonimi)
# LE SEGUENTI FUNZIONI POSSONO ESSERE USATE SIA PER I MODELLI DI WORD2VEC CHE PER QUELLI ALLINEATI DI CADE

#models = [] #per caricare i modelli pre-addestrati
#for k in range(n_mod):
  #model = Word2Vec.load(text+'_'+str(k)+'.model')
  #models.append(model)


def find_similar(word: str, n_similar: int, n_mod:int, pos_tag_prefix: str, models: list) -> list: #parola da analizzare, numero di parole simili volute, numero modelli, prefisso POS 
                                                                                                   #desiderato, lista dei modelli

  most_similar = dict()

  for k in range(n_mod):
    #ottengo la lista delle n parole simili alla parola target per modello, e filtro mantenendo solo quelle che hanno il POS tag desiderato
    similar = models[k].wv.most_similar(word, topn=(5*n_similar)) #moltiplico il numero di parole per avere un campione più ampio
    similar_filtered = list(filter(lambda similar: check_pos_tag(similar[0], pos_tag_prefix), similar))

    #combino le parole ottenute dai vari modelli
    for (string, similarity) in similar_filtered:
      if string not in most_similar.keys():
        most_similar[string] = similarity
      else:
        most_similar[string] = max(most_similar[string], similarity)

  #ordino le parole in base alla similarità e seleziono solo il numero desiderato
  similar_filtered_sorted = sorted(most_similar.items(), key=lambda item: item[1], reverse=True)[:n_similar]
  similar_final = [w for (w, sim) in similar_filtered_sorted]

  return similar_final





# Funzione utile in quella precedente, che definisce il "part-of-speech" tag
# tagger = treetaggerwrapper.TreeTagger(TAGLANG='it')

def check_pos_tag(word: str, tag_prefix: str) -> bool:
  tag = tagger.tag_text(word)
  part_of_speech = treetaggerwrapper.make_tags(tag)
  return part_of_speech[0].pos.startswith(tag_prefix)


# Declinazione della funzione generale per trovare i sinonimi

def find_synonyms(word: str, n_synonyms: int, n_mod: int, models: list) -> list:
  return find_similar(word, n_synonyms, n_mod, 'NOM', models) #sistemare, se necessario, il tag

# Declinazione della funzione generale per trovare gli aggettivi descrittivi

def find_describing_adjectives(words: list, n_adjectives: int, n_mod: int, models: list) -> None: #in questo caso passo la lista contenente i sinonimi
  for word in words:
    print(find_similar(word, n_adjectives, n_mod, 'ADJ', models))

### TEST FUNZIONI

In [16]:
# Test funzione di training WORD2VEC

training_W2V(frasi_decameron, "decameron", 5)
training_W2V(frasi_orlando, "orlandofurioso", 5)

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [19]:
# Test funzione di training CADE

!cat Decameron.txt Orlando\ furioso.txt > compass.txt
lista_testi = ['Decameron', 'Orlando furioso']

training_CADE(lista_testi, 5)

Training the compass from scratch.


/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Training embeddings: slice Decameron.txt.
Initializing embeddings from compass.
Training embeddings: slice Orlando furioso.txt.
Initializing embeddings from compass.
Training the compass from scratch.
Initializing embeddings from compass.
Training embeddings: slice Decameron.txt.
Initializing embeddings from compass.
Training embeddings: slice Orlando furioso.txt.
Initializing embeddings from compass.
Training the compass from scratch.
Initializing embeddings from compass.
Training embeddings: slice Decameron.txt.
Initializing embeddings from compass.
Training embeddings: slice Orlando furioso.txt.
Initializing embeddings from compass.
Training the compass from scratch.
Initializing embeddings from compass.
Training embeddings: slice Decameron.txt.
Initializing embeddings from compass.
Training embeddings: slice Orlando furioso.txt.
Initializing embeddings from compass.
Training the compass from scratch.
Initializing embeddings from compass.
Training embeddings: slice Decameron.txt.
In

In [29]:
# Test funzione parole simili per topos

df = similar_words_per_topos("donna", 5, 10, "Decameron")
df2 = similar_words_per_topos("donna", 5, 10, "Orlandofurioso")
print(df)
print(df2)

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)  \
0       gentil              5        0.975001               0.134948   
1    figliuola              5        0.955136               0.139902   
2        forte              5        0.951564               0.137129   
3    ricciardo              5        0.945880               0.008638   
4      rinaldo              4        0.927420               0.655332   
5       moglie              5        0.934323               0.593367   
6        padre              1        0.929289               1.000000   
7        buona              1        0.925428               0.302578   
8       pietro              2        0.929491               0.105911   
9   maravigliò              3        0.920242               0.103417   
10     giovane              3        0.923469               0.049917   
11   cavaliere              2        0.925268               0.027037   
12      udendo              1        0.924786               0.29

In [37]:
# Test funzione per comparazione parole fra embeddings
# In questo caso prendo come riferimento il Decameron, da cui estraggo il vettore della parola target "donna", e confronto con i vettori dell'altro testo

df3 = compare_word_btw_embeddings("donna", 5, "Decameron", "Orlando furioso")
print(df3)

# Ora provo il contrario

df4 = compare_word_btw_embeddings("donna", 5, "Orlando furioso", "Decameron")
print(df4)

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


        Words  Med.similarity
7       donna        0.801823
17     regina        0.634103
19    vecchia        0.631947
18    sorella        0.568712
0   Discordia        0.508708
1    Doralice        0.500033
9        fata        0.498671
6    difender        0.497261
11    giovane        0.492326
14       maga        0.487076


/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


         Words  Med.similarity
5        donna        0.801823
15  giovinetta        0.653398
1      Bernabò        0.643578
13       giace        0.635454
10      fante,        0.581835
4      domanda        0.543919
19       monna        0.543856
11  fanticella        0.536388
14     giovane        0.530860
3    cameriera        0.530698


In [48]:
# Test funzione analogie

df5 = compute_analogies("decameron", 5, "uomo", "guerra", "amore")
print('DECAMERON: uomo-amore+guerra')
print(df5)

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


DECAMERON: uomo-amore+guerra
            Cosine_similarity
Words                        
chiamato             0.935694
nome                 0.904417
gentile              0.888838
ricco                0.884999
messer               0.880452
torello              0.837737
buono                0.831476
mercatante           0.827906
adunque              0.820707
figliuolo            0.819537
chiamata             0.817307
guiglielmo           0.806817


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app


In [ ]:
# Funzione che mergia due file di testo sfruttando la secure hashes. La tengo qua in caso serva (ho usato la procedura che aveva aggiunto Gian
# in quanto andava ad aggiungere le stopwords direttamente al file della libreria NTLK, oltre che risultare molto più semplice)

# Procedura che combina n files in un unico file non considerando i duplicati

def get_sha1(file):
    checksum = hashlib.sha1()
    for chunk in iter(lambda: file.read(4096), b""):
        checksum.update(chunk)
    return checksum.hexdigest()

def already_copied(file, checksums):
    checksum = get_sha1(file)
    if checksum not in checksums:
        checksums.add(checksum)
        return False
    return True

checksums = set()
with open("Output/stopwords_merged.txt", "wb") as merged:
    for file in glob.glob("Output/stop*.txt"):
        with open(file, "rb") as file:
            if already_copied(file, checksums):
                continue
            file.seek(0) # Ritorno all'inizio del file
            for line in file:
                merged.write(line)